# Calibrating LASER-based Models with Optuna and Coiled

In this notebook, we'll show you how to calibrate a LASER-based model using [Optuna](https://optuna.org/), and scaling the calibration to the cloud using [Coiled](https://www.coiled.io/).

The information from the notebook was derived form the following sources:
- [Coiled Hyperparameter Optimization w/Optuna Example](https://docs.coiled.io/user_guide/hpo.html)
- [Detailed Coiled+Optuna Examples](https://docs.coiled.io/user_guide/xgboost-hpo.html)
- [GitHub Repository for Coiled+Optuna Examples](https://github.com/coiled/dask-xgboost-nyctaxi)
- [Starsim Coiled Exploration](https://github.com/starsimhub/starsim_compute_scaling/tree/main/platforms)
- [Katherine's Ax Example](https://github.com/krosenfeld-IDM/sandbox-botorch/blob/main/laser/london/run_service.py)
- [Optuna with K8s (for comparison)](https://github.com/optuna/optuna-examples/tree/main/kubernetes/simple)

### 1. Set up the cluster

In [ ]:
import coiled

cluster = coiled.Cluster(n_workers=20, name="laser-coiled-optuna")
client = cluster.get_client()

### 2. Set up the study

In [ ]:
import optuna

backend_storage = optuna.storages.InMemoryStorage()
dask_storage = optuna.integration.DaskStorage(storage=backend_storage)

study = optuna.create_study(
    direction="maximize",
    storage=dask_storage,
    sampler=optuna.samplers.RandomSampler(),
)

In [ ]:
def objective(trial):
    # The objective function should include the following steps:
    # 1. extract trial parameters from the trial object
    # 2. run the simulation with the trial parameters
    # 3. return the simulation score

    return 1

### 3. Run the optimization in parallel

In [ ]:
%%time

from dask.distributed import wait

futures = [
    client.submit(study.optimize, objective, n_trials=1, pure=False) for _ in range(500)
]

_ = wait(futures)

### 4. Evaluate the results

In [ ]:
print(study.best_params)
print(study.best_value)

In [ ]:
from optuna.visualization.matplotlib import plot_optimization_history

plot_optimization_history(study);

### 5. Clean up

In [ ]:
cluster.shutdown()